In [0]:
!git clone https://github.com/BurguerJohn/3d-ken-burns.git

In [0]:
cd /content/3d-ken-burns

In [0]:
!sh download.bash

In [0]:
import os
os.environ['CUDA_HOME'] = "/usr/local/cuda"
print(os.environ['CUDA_HOME'])

#Convert photos
Take all images from path and apply the zoom effect.

In [0]:
#@markdown Path with all the photos:
path = "/content/photos" #@param{type:"string"}
#@markdown Zoom configuration:
centerU = 0.5 #@param{type:"slider", min:0, max:1, step:0.1}
centerV = 0.5 #@param{type:"slider", min:0, max:1, step:0.1}
easyZoom = 1 #@param{type:"slider", min:0, max:1, step:0.1}
turbulence = 0 #@param{type:"slider", min:0, max:1, step:0.1}
#@markdown More zoom configuration:
maximumZoom = 1.25 #@param{type:"number"}
shift = 100 #@param{type:"number"}

files = []

for r, d, f in os.walk(path):
  for file in sorted(f):
    files.append(os.path.join(r, file))


for i in files:
  newName = i.replace(" ", "_")
  newName = newName.replace("(", "_")
  newName = newName.replace(")", "_")
  os.rename(i, newName) 
  i = newName

  name = os.path.basename(i)
  dirr = os.path.dirname(i)
  name_no_ext = name.split('.')[0]
  !mkdir {dirr}_mp4

  outPath = dirr + "_mp4/" + name_no_ext+".mp4"
  outPathCorr = dirr + "_mp4/" +  name_no_ext+ "_pad.mp4"
  !python autozoom.py --in {i} --out {outPath} --centerU {centerU} --centerV {centerV} --shift {shift} --zoom {maximumZoom} --easy {easyZoom} --turbulence {turbulence}
  !ffmpeg -y -i {outPath} -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -crf 17 -pix_fmt yuv420p {outPathCorr}
  !rm {outPath}